In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn


In [2]:
read_data = pd.read_csv('model.csv')
read_test = pd.read_csv('predict_test.csv')

In [3]:
read_data = read_data.drop('Unnamed: 0', axis = 1)
read_test = read_test.drop('Unnamed: 0', axis = 1)

In [4]:
read_data.shape, read_test.shape

((95662, 20), (45019, 19))

In [5]:
y = read_data['FraudResult']
x = read_data.drop('FraudResult', axis = 1)

In [6]:
sum(y)/y.shape[0], sum(y), y.shape[0]

(0.00201752001839811, 193, 95662)

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
#min_max = MinMaxScaler()

#train_split =std.fit_transform(x)

#test_data = std.fit_transform(read_test)

In [8]:
#train_split = min_max.fit_transform(train_split)
#test_data = min_max.fit_transform(test_data)

In [59]:
from sklearn.feature_selection import SelectKBest, chi2

#X_new = SelectKBest(chi2, 10).fit_transform(train_split, y)
#X_new = SelectKBest(chi2, 10).fit_transform(x, y)

In [22]:
from sklearn.model_selection import train_test_split
y = y.astype('int')
x_train, x_test,y_train, y_test = train_test_split(x, y, train_size = 0.8, random_state = 42)


In [61]:
# concatenate our training data back together
X = pd.concat([x_train, y_train], axis=1)
X.columns

Index(['Value', 'PricingStrategy', 'ProductCategory_airtime',
       'ProductCategory_data_bundles', 'ProductCategory_financial_services',
       'ProductCategory_movies', 'ProductCategory_other',
       'ProductCategory_ticket', 'ProductCategory_transport',
       'ProductCategory_tv', 'ProductCategory_utility_bill',
       'ProviderId_ProviderId_1', 'ProviderId_ProviderId_2',
       'ProviderId_ProviderId_3', 'ProviderId_ProviderId_4',
       'ProviderId_ProviderId_5', 'ProviderId_ProviderId_6',
       'transaction_month', 'transaction_day', 'FraudResult'],
      dtype='object')

In [69]:
from sklearn.utils import resample

# separate minority and majority classes
not_fraud = X[X.FraudResult==0]
fraud = X[X.FraudResult==1]

# upsample minority
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_fraud, fraud_upsampled])

# check new class counts
upsampled.FraudResult.value_counts()


1    76372
0    76372
Name: FraudResult, dtype: int64

In [70]:
#from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression()
#from sklearn.metrics import accuracy_score, precision_score, f1_score
#lr.fit(x_train, y_train)
#lr.score(x_train, y_train), lr.score(x_test, y_test)

In [71]:
#pred = lr.predict(x_test)

#f1_score(pred, y_test), precision_score(pred, y_test)

In [75]:
#from imblearn.over_sampling import SMOTE

#sm = SMOTE(random_state=12, ratio = 1.0)
#x_train_res, y_train_res = sm.fit_sample(x_train, y_train)


In [76]:
y_train_res = upsampled['FraudResult']
x_train_res = upsampled.drop('FraudResult', axis = 1)

In [77]:
x_train_res.shape

(152744, 19)

In [78]:
#test_data.shape

In [79]:
#lr.fit(x_train_res, y_train_res)
#lr.score(x_train_res, y_train_res)

In [80]:
#pred = lr.predict(x_test)
#f1_score(pred, y_test)

In [81]:
#precision_score(pred, y_test)

In [82]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train_res, y_train_res)
pred = knn.predict(x_test)

print('confusion matrix')
print(metrics.confusion_matrix(y_test, pred))


confusion matrix
[[19058    39]
 [    0    36]]


In [83]:
print('classification report')
print(metrics.classification_report(y_test, pred))


classification report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19097
           1       0.48      1.00      0.65        36

    accuracy                           1.00     19133
   macro avg       0.74      1.00      0.82     19133
weighted avg       1.00      1.00      1.00     19133



In [84]:

print('Accuracy: %f' %(metrics.accuracy_score(y_test, pred)))


print('Area under the curve : %f' % (metrics.f1_score(y_test, pred)))



Accuracy: 0.997962
Area under the curve : 0.648649


In [90]:
dict_score = {}
for k in [1,2,3,4,5,6,7,8,9,10]:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(x_train_res, y_train_res)
    pred = knn.predict(x_test)
    score = f1_score(pred, y_test)
    dict_score[str(k)] = score

dict_score

{'1': 0.6796116504854368,
 '10': 0.5581395348837209,
 '2': 0.6796116504854368,
 '3': 0.6486486486486487,
 '4': 0.6486486486486487,
 '5': 0.6260869565217391,
 '6': 0.6260869565217391,
 '7': 0.576,
 '8': 0.5806451612903226,
 '9': 0.5625}

In [96]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#rand = RandomForestClassifier()
rand = GradientBoostingClassifier()
rand.fit(x_train_res, y_train_res)
pred = rand.predict(x_test)
f1_score(pred, y_test)

0.5625

In [98]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier()
learning_rate = [0.1,1, 10]

n_estimators = [100,200,400]

param_grid = dict(learning_rate = learning_rate,
                 n_estimators = n_estimators)

grid = GridSearchCV(estimator=gbrt,
                    param_grid=param_grid,
                    scoring='roc_auc',
                    verbose=1,
                    n_jobs=-1)
grid_result = grid.fit(x_train_res, y_train_res)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 34.5min finished


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\user\\AppData\\Local\\Temp\\joblib_memmapping_folder_4648_9648533725\\4648-756980624912-575100a971b849c1bb929d7644dda88e.pkl'

In [92]:

print('Accuracy: %f' %(metrics.accuracy_score(y_test, pred)))


print('Area under the curve : %f' % (metrics.f1_score(y_test, pred)))


Accuracy: 0.997909
Area under the curve : 0.583333


In [93]:
print('classification report')
print(metrics.classification_report(y_test, pred))


classification report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19097
           1       0.47      0.78      0.58        36

    accuracy                           1.00     19133
   macro avg       0.73      0.89      0.79     19133
weighted avg       1.00      1.00      1.00     19133



In [46]:

print('confusion matrix')
print(metrics.confusion_matrix(y_test, pred))


confusion matrix
[[19097     0]
 [    0    36]]


In [86]:
prediction = knn.predict(read_test)
#prediction = rand.predict(read_test)

In [87]:
ids = pd.read_csv('TestId.csv')
ids.head()

,Unnamed: 0,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,ProductId,ChannelId
0,0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,ProductId_3,ChannelId_3
1,1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,ProductId_15,ChannelId_3
2,2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,ProductId_6,ChannelId_2
3,3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,ProductId_10,ChannelId_3
4,4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,ProductId_6,ChannelId_2


In [88]:
test = pd.DataFrame()
test['TransactionId'] = ids['TransactionId']
test['FraudResult'] =prediction.astype('int')

test.to_csv('rand.csv', index = False)

In [89]:
test.shape

(45019, 2)